In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

    
# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    # グラフのインスタンスを作成
    graph_builder = StateGraph(State)

    # 言語モデルの定義
    llm = ChatOpenAI(model_name=model_name)

    # 変更点：ツール定義の紐づけ
    llm_with_tools = llm.bind_tools(tools)

    # グラフにチャットボットノードを追加
    graph_builder.add_node("chatbot", lambda state: {"messages": [llm_with_tools.invoke(state["messages"])]})

    # ツールノードの作成
    tool_node = ToolNode(tools)

    # グラフにツールノードを追加
    graph_builder.add_node("tools", tool_node)

    # 条件付エッジの作成
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition, # ツール呼出と判断したらツールノードを呼ぶ
    )

    # ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
    # ツールからチャットボットへの戻りエッジを作成
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノードの指定
    graph_builder.set_entry_point("chatbot")

    # 記憶を持つ実行可能なステートグラフの作成
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)

    return graph


# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)

こんにちは
こんにちは！今日はどのようにお手伝いできますか？
1たす2は？
1たす2は3です。
台湾観光について検索結果を教えて

[{"url": "https://www.agu.ac.jp/pdf/graduate/thesis/letters1/35-3.pdf", "content": "「視察団」として台湾に来たと考 えられる。その場合何を視察にきたのかという疑問が生じる。1930 年の霧社事件2後、 台湾と日本との関係は平和な道へ戻り、交流の人数は以前より大幅に上昇した。その 後、 張倩容の研究によれば、 台湾は日本植民地時期に観光制度が成立し、 1935 年の 「始 政四十周年記念台湾博覧会」は台湾観光の地位を確定したとされる3。しかし、私見を 述べれば、植民地時代の台湾における観光は単純な観光地の旅行や訪問のみならず、 資源開発などの経済的な人的・物的の交流までも内包するものである。そこで、本章 では、台湾と日本の人的交流の視点から、1920 年から1930 年代までを中心に台湾に おける観光事業は如何に変遷したのかを考察する。具体的には、日本政府と台湾総督 府の観光政策、交通政策、及び渡台日本人の台湾観光に与えた影響について検討して いく。 8 第一節 台湾旅行・観光の歴史背景 近年、台湾における観光についての研究が相次いで発表されている。そのなかで日 本人学者曽山毅の『植民地台湾と近代ツーリズム』という著書があり、台湾観光に関 する研究において先駆的役割を果たしている。曽山は台湾の鉄道建設は植民地支配者 の政治と経済を結びつける手段の一つだと強調している4。その背景で台湾の観光を考 えると、植民地統治のための一つの統制手段であると言えないこともない。曽山が既 に触れているが、台湾の鉄道建設は重要なものであることは明白である。その点に関 して蔡龍保の『推動時代巨輪：日治中期臺灣國有鐵路1910－1936』は台東県、宜蘭県 など新たな海岸線鉄道建設の開発により、経済的利益を誘発し、それと同時に観光事 業によって、地方を発展させたと論じた5。しかし、台湾の鉄道が観光事業に対してど のような役割を果たしたのかについては十分に検討されていない。 [...] 筈ナルカ横浜上陸団員ニ対シテ同地商業会議所ニ依頼シ之ヲ配布セムトノ計 画シツゝアリ 右及申（通）報